In [722]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [829]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException

import pandas as pd
import time
#brew install geckodrivee

In [830]:
def error_parse(error):
    if "trustee" in error:
        return "CUTrustee"
    return error

def email_chars(email):
    if email:
        if '•' in email:
            sublen = len(email.split("@")[0])-2
            return email.split("@")[0][:2]+"?"*sublen+"@"+email.split("@")[1]
    return email

In [856]:
driver = webdriver.Firefox()

In [823]:
###to shut things down:
# elem.clear()
# elem.send_keys("pycon")
# elem.send_keys(Keys.RETURN)
# assert "No results found." not in driver.page_source
# driver.close()

###getting other things done:
# dir(By)
# print(driver.title)
# print(dir(driver))
# 
# ids = elem.find_elements_by_xpath('//*[@id]')

#re.search(r"\d", string) -- find first digit in the string

##getting the components on the page:
# ids = elem.find_elements_by_xpath('//*[@id]')
# for ii in ids:
#     #print ii.tag_name
#     print(ii.get_attribute('id'))

In [857]:
driver.get("https://tools.alumni.columbia.edu/uni/SelectUser.aspx")

In [858]:
elem = driver.find_element_by_id("container")

In [859]:
firstname = elem.find_elements_by_id("ctl00_rightcol_ph_first_tb")

In [860]:
lastname = elem.find_elements_by_id("ctl00_rightcol_ph_last_tb")
searchbutton=elem.find_elements_by_id("ctl00_rightcol_ph_Button2")
school=elem.find_elements_by_id("ctl00_rightcol_ph_school_ddl_Input")

In [ ]:
# alumni = []
# hiccup_letters = []
# hiccup_emails = []
for class_year in range(1976,1990):
    for firstchar in "abcdefghijklmnopqrstuvwxyz":
        for lastchar in "abcdefghijklmnopqrstuvwxyz":
            try:
                school=elem.find_elements_by_id("ctl00_rightcol_ph_school_ddl_Input")
                time.sleep(1)
                school[0].click()
                college=driver.find_element_by_id("ctl00_rightcol_ph_school_ddl_c6").click()
                year = elem.find_element_by_id("ctl00_rightcol_ph_year_ddl")
                year.click()
                year.send_keys(str(class_year))

                firstname[0].send_keys(firstchar)
                lastname[0].send_keys(lastchar)
                searchbutton[0].send_keys(Keys.ENTER)
                time.sleep(3)
                try:
                    table = driver.find_element_by_id("ctl00_rightcol_ph_result_grid")
                except NoSuchElementException:
                    time.sleep(5)
                    try:
                        table = driver.find_element_by_id("ctl00_rightcol_ph_result_grid")
                    except NoSuchElementException:
                        time.sleep(15)
                        table = driver.find_element_by_id("ctl00_rightcol_ph_result_grid")

                people = table.find_elements(By.TAG_NAME, "tr")
                if len(people)>1:

                    for person in people[1:]:

                        info = person.text
                        name = info.split("CC ")[0][:-4]
                        other = "".join(info.split("'")[1:])
                        alumni.append([name,other,int(str(class_year)[2:]),"CC"])

                    for record in alumni[(-len(people)+1):]:
                        try:
                            link = driver.find_element_by_link_text(record[0])
                            link.click()
                        except:
                            driver.back()
                            link = driver.find_element_by_link_text(record[0])
                            link.click()
                        try:
                            time.sleep(2)
                            email_table=driver.find_element_by_id("ctl00_leftcol_ph_emails_rbl")
                            email_list = email_table.find_elements(By.TAG_NAME, "tr")

                            for emails in email_list:
                                time.sleep(1)
                                record.append(email_chars(emails.text))
                                if len(record[-1])<2:
                                    time.sleep(2)
                                    record[-1]=email_chars(emails.text)
                                    if len(record[-1])<2:

                                        #if the email isn't copying into the list of emails:
                                        raise Exception()
                                        print("Check hiccup emails for email that isn't working for "+str([person.text,emails.text]))
                                        hiccup_emails.append([person.text,emails.text])

                            driver.back()

                        #if no emails come up:
                        except NoSuchElementException:
                            #record.append(email)
                            try:
                                error_message = driver.find_element_by_id("ctl00_rightcol_ph_ErrorPnl")
                                if type(error_message)!=str:
                                    error_message = error_message.text
                                record.append(error_parse(error_message))
                            except NoSuchElementException:
                                error_message = "other error"
                                record.append(error_message)
                                driver.back()
                                continue

            except Exception:
                print("hiccup on:")
                print(Exception)
                print(firstchar)
                print(lastchar)
                hiccup_letters.append([firstchar,lastchar])       
            
            attempts = 0
            while attempts < 10:
                try:
                    driver.get("https://tools.alumni.columbia.edu/uni/SelectUser.aspx")
                    break
                except:
                    driver.refresh()
                    time.sleep(5)
                    attempts = attempts + 1
                    continue
                
            if attempts == 10:
                print("SOMETHING WRONG: 10 ATTEMPTS & NOTHING FIXED")
                
            elem = driver.find_element_by_id("container")
            firstname = elem.find_elements_by_id("ctl00_rightcol_ph_first_tb")
            lastname = elem.find_elements_by_id("ctl00_rightcol_ph_last_tb")
            searchbutton=elem.find_elements_by_id("ctl00_rightcol_ph_Button2")
            school=elem.find_elements_by_id("ctl00_rightcol_ph_school_ddl_Input")
            
    pd.DataFrame(alumni).to_csv("CC_"+str(class_year)+"_alumni.csv")
    alumni = []
            

hiccup on:
<class 'Exception'>
j
m
hiccup on:
<class 'Exception'>
y
p
hiccup on:
<class 'Exception'>
i
p
hiccup on:
<class 'Exception'>
j
s
hiccup on:
<class 'Exception'>
m
s
hiccup on:
<class 'Exception'>
p
r
hiccup on:
<class 'Exception'>
r
e
hiccup on:
<class 'Exception'>
k
l
hiccup on:
<class 'Exception'>
m
s
hiccup on:
<class 'Exception'>
r
k
hiccup on:
<class 'Exception'>
s
k
hiccup on:
<class 'Exception'>
a
m
hiccup on:
<class 'Exception'>
j
m
hiccup on:
<class 'Exception'>
n
z
hiccup on:
<class 'Exception'>
t
j
